In [1]:
from pystruct.models import LatentNodeCRF
from pystruct.models import GraphCRF
from pystruct.learners import FrankWolfeSSVM
from pystruct.learners import OneSlackSSVM
from pystruct.learners import NSlackSSVM
from pystruct.learners import LatentSSVM
from pystruct.learners import SubgradientLatentSSVM
from pystruct.datasets import load_letters
from pystruct.utils import make_grid_edges
import numpy as np
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split

## Convert seqs file into a character array

In [2]:
msa = np.array([np.array(list(x), dtype=object) for x in open('adk_lid_uppercase.seqs', 'r').read().split('\n') if x], dtype=object)
default_len = len(msa[0])

## Load controls

In [3]:
df = pd.read_csv('control_seqs.csv')

## Convert controls into a character array

In [4]:
controls = np.array([list(i[0:default_len]) for i in df.seq if len(i) >= default_len], dtype=object)

## Combine msa and controls

In [5]:
combination = np.concatenate((controls, msa))

In [6]:
#create library of all amino acids
library = {'-': 0, 'C': 1, 'D': 2, 'S': 3,
 'Q': 4, 'K': 5, 'I': 6, 'P': 7,
 'T': 8, 'F': 9, 'N': 10, 'G': 11,
 'H': 12, 'L': 13, 'R': 14, 'W': 15,
 'A': 16, 'V': 17, 'E': 18, 'Y': 19,
 'M': 20, 'X': 21}

len(library)

22

### Set all latent edges so that the final output is a 1D array

In [7]:
l = set(range(default_len))
latent_edges = np.array([list(i) for i in itertools.combinations(l, 2)])
latent_edges.shape

(1378, 2)

In [8]:
#one hot encode msa values
X = []
for i in combination:
    node = []
    for k in i:
        z = np.zeros(len(library), dtype=int)
        z[library[k]] = 1
        node.append(z)
    X.append(np.array(node))
X=np.array(X)
X.shape

(672, 53, 22)

In [9]:
X_ = list(zip(X, [latent_edges for _ in X], [default_len-1 for _ in X]))

# X, latent_edges per sample, Number of Hidden nodes
# number of hidden nodes reduced the msa x length down to one dimension

In [10]:
y_controls = np.zeros(shape = (len(controls), 1), dtype=int)
y_msa = np.ones(shape = (len(msa), 1), dtype=int)
y = np.concatenate((y_controls, y_msa))

In [11]:
print(y_controls.shape)
print(y_msa.shape)


(373, 1)
(299, 1)


In [12]:
# split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=.2)

In [13]:
print(len(X_train)) # out of the total controls and targets, this was used to train the model
print(len(X_test))


537
135


In [14]:
len(library) # library of amino acids == features

22

In [15]:
model = LatentNodeCRF(n_labels=2, n_features=len(library), latent_node_features=True)

In [16]:
ssvm = OneSlackSSVM(model, max_iter=200, C=100, n_jobs=-1, show_loss_every=10, inference_cache=50)

latentssvm = LatentSSVM(base_ssvm=ssvm)

In [17]:
import time
tic = time.time()
latentssvm.fit(X_train, y_train)
tic2 = time.time() 
runtime = tic2 - tic
print(f'Runtime for training: {runtime}s')

/usr/local/lib/python3.8/dist-packages/pystruct-0.3.2-py3.8-linux-x86_64.egg/pystruct/learners/one_slack_ssvm.py:307: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.all(y_1 == y_2)


Runtime for training: 527.684656381607s


In [18]:
latentssvm.score(X_test, y_test)

1.0

In [19]:
pd.options.display.max_rows = 200

In [20]:
pd.DataFrame([latentssvm.predict(X_test), y_test]).T.head()


,0,1
0,[0],[0]
1,[1],[1]
2,[0],[0]
3,[1],[1]
4,[0],[0]


# 99.26% accuracy!

In [21]:
latentssvm.w
#?????? Not sure what this is

array([ 1.50523204e-01,  0.00000000e+00,  0.00000000e+00,  1.65709608e-04,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -3.00869073e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.50124925e-01,  0.00000000e+00, -1.50523204e-01,  0.00000000e+00,
        0.00000000e+00, -1.65709608e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.00869073e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.50124925e-01,  0.00000000e+00,
       -3.50475862e-03,  1.72347410e-01,  2.43282401e-01,  2.38457845e-03,
       -1.33146341e-01, -3.45177162e-02,  1.12028240e-02,  2.80931725e-01,
       -5.84009676e-02, -